In [78]:
import pandas as pd
import os

# 1. Get the current folder where the notebook is running
current_folder = os.getcwd()

# 2. Join it with your filename
file_path = os.path.join(current_folder, "Customer Call List.xlsx")

# 3. Load the data
df = pd.read_excel(file_path)

DATA EXPLORATION AND FORMATTING COLUMN HEADER

In [79]:
# First, we take a look at the data info to know what we are dealing with.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   CustomerID         21 non-null     int64 
 1   First_Name         21 non-null     object
 2   Last_Name          20 non-null     object
 3   Phone_Number       19 non-null     object
 4   Address            21 non-null     object
 5   Paying Customer    21 non-null     object
 6   Do_Not_Contact     17 non-null     object
 7   Not_Useful_Column  21 non-null     bool  
dtypes: bool(1), int64(1), object(6)
memory usage: 1.3+ KB


In [95]:
# Now , we see the first few rows of the dataframe
# not running df.head() as this dataset is small, so no problem showing entire dataset.
df

,customer_id,first_name,last_name,phone_number,address,paying_customer,do_not_contact
0,1001,Frodo,Baggins,123-545-5421,"123 Shire Lane, Shire",Yes,No
1,1002,Abed,Nadir,123-643-9775,93 West Main Street,No,Yes
2,1003,Walter,White,NaN,298 Drugs Driveway,N,NaN
3,1004,Dwight,Schrute,123-543-2345,"980 Paper Avenue, Pennsylvania, 18503",Yes,Y
4,1005,Jon,Snow,876-678-3469,123 Dragons Road,Y,No
5,1006,Ron,Swanson,304-762-2467,768 City Parkway,Yes,Yes
6,1007,Jeff,Winger,NaN,1209 South Street,No,No
7,1008,Sherlock,Holmes,876-678-3469,98 Clue Drive,N,No
8,1009,Gandalf,NaN,NaN,123 Middle Earth,Yes,NaN
9,1010,Peter,Parker,123-545-5421,"25th Main Street, New York",Yes,No


In [81]:
# I want to stadardize the column names, where space is replaced with underscore,
# all letters are in lowercase, and no leading or trailing spaces.
df.columns = df.columns.str.lower().str.replace(' ', '_').str.strip()

In [82]:
# Run this to see the updated column names
df.columns

Index(['customerid', 'first_name', 'last_name', 'phone_number', 'address',
       'paying_customer', 'do_not_contact', 'not_useful_column'],
      dtype='object')

In [83]:
# I kinda dislike the column name 'customerid', so let's rename it to 'customer_id for better readability.
df = df.rename(columns={'customerid': 'customer_id'})

In [84]:
# Dropping 'not_useful_column' as it says not useful column, duhh.
# Also dropping duplicate rows. 
df = df.drop_duplicates().drop(columns='not_useful_column')


FORMATTING 'last_name' AND 'phone_number' WITH REGEX

In [89]:
# This format 'last_name' column by removing special characters and leading/trailing spaces
df['last_name'] = df['last_name'].str.replace(r'[^a-zA-Z0-9 ]', ' ', regex=True)

In [90]:
# For 'phone_number' column, remove everything that is NOT a number
df['phone_number'] = df['phone_number'].str.replace(r'[^\d]', '', regex=True)

In [91]:
# Use apply function to apply to each rows of the 'Phone_Number' column.
# Use lambda , as a shortcut to define a function in one line. 
# Every x need to be converted to string first.
# Then we slice the string accordingly to its position and add dashes in between.
df['phone_number'] = df['phone_number'].apply(lambda x: str(x)[0:3] + '-' + str(x)[3:6] + '-' + str(x)[6:10])


In [93]:
# We realized that some 'Phone_Number' values are invalid, like 'nan--' or '--'.
# Let's see the value counts first.
print(df['phone_number'].value_counts(dropna=False))

phone_number
nan--           5
876-678-3469    4
123-545-5421    3
123-643-9775    2
123-543-2345    2
304-762-2467    2
--              2
Name: count, dtype: int64


In [94]:
# From value counts, we can see there are some invalid phone numbers like 'nan--' and '--'.
# Replaces both 'nan--' and '--' with an np.nan value
import numpy as np
df['phone_number'] = df['phone_number'].replace(['nan--', '--'], np.nan, regex=False)

PARSING 'address' INTO THREE COLUMNS 'street_name','state', 'zip_code'

In [97]:
# split 'address' column into three new columns: 'street_name', 'state', and 'zip_code'
df[['street_name','state','zip_code']]=df['address'].str.split(',',n=2,expand=True)

In [99]:
# after parsed 'address' we found that 'state' and 'zipcode' contain 'None' value.
# We will replace those 'None' string values with np.nan
# We cant use replace here because 'None' is not a special value but a string.
df[['state', 'zip_code']] = df[['state', 'zip_code']].fillna(np.nan)

In [100]:
# Finally, drop the original 'address' column
df = df.drop(columns=['address'])

STANDARIZE 'paying_customer' AND 'do_not_contact' VALUE INTO 'Yes' AND 'No'

In [104]:
# See, for 'paying_customer' and 'do_not_contact' columns, it supposed to be values like 'Yes' and 'No' only.
# But there are some other values like 'Y', 'N', 'N/a'.
# So, we will standardize these columns to have only 'Yes', 'No', and np.nan values.
# let's first see the current value counts for these two columns
print(df['paying_customer'].value_counts(dropna=False))
print("-" * 30) # this is separator line
print(df['do_not_contact'].value_counts(dropna=False))

paying_customer
Yes    13
No      6
NaN     1
Name: count, dtype: int64
------------------------------
do_not_contact
No     12
Yes     4
NaN     4
Name: count, dtype: int64


In [103]:
# Now, we will replace those values accordingly.

df[['paying_customer', 'do_not_contact']] = df[['paying_customer', 'do_not_contact']].replace(
    {
     'paying_customer': {'Y': 'Yes', 'N': 'No', 'N/a': np.nan},
      'do_not_contact': {'Y': 'Yes', 'N': 'No'}
})

In [105]:
# We are done with data cleaning. Let's see the final cleaned
df

,customer_id,first_name,last_name,phone_number,paying_customer,do_not_contact,street_name,state,zip_code
0,1001,Frodo,Baggins,123-545-5421,Yes,No,123 Shire Lane,Shire,NaN
1,1002,Abed,Nadir,123-643-9775,No,Yes,93 West Main Street,NaN,NaN
2,1003,Walter,White,NaN,No,NaN,298 Drugs Driveway,NaN,NaN
3,1004,Dwight,Schrute,123-543-2345,Yes,Yes,980 Paper Avenue,Pennsylvania,18503
4,1005,Jon,Snow,876-678-3469,Yes,No,123 Dragons Road,NaN,NaN
5,1006,Ron,Swanson,304-762-2467,Yes,Yes,768 City Parkway,NaN,NaN
6,1007,Jeff,Winger,NaN,No,No,1209 South Street,NaN,NaN
7,1008,Sherlock,Holmes,876-678-3469,No,No,98 Clue Drive,NaN,NaN
8,1009,Gandalf,NaN,NaN,Yes,NaN,123 Middle Earth,NaN,NaN
9,1010,Peter,Parker,123-545-5421,Yes,No,25th Main Street,New York,NaN
